In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader,SubsetRandomSampler
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from common import setup_dataflow,EEG,getepoch,do_plot,create_dataloader
import wandb
from mne.datasets import sample

In [2]:
from model import ConvNet,CNN2D,ConvNet_physionet
from torchsummary import summary
net = ConvNet().cuda()


In [3]:
# execute
#net.load_state_dict(torch.load('./save_weight/sunsun/sunsun_85.1852'))

# MI
net.load_state_dict(torch.load('./save_weight/Beau_S48_MI/0.6492_Beau_S48_MI_0.6492_82.6087'))

<All keys matched successfully>

In [4]:
parms = net.state_dict()
parms.keys()

odict_keys(['layer1.0.weight', 'layer1.0.bias', 'layer1.1.weight', 'layer1.1.bias', 'layer1.1.running_mean', 'layer1.1.running_var', 'layer1.1.num_batches_tracked', 'layer2.0.weight', 'layer2.0.bias', 'layer2.1.weight', 'layer2.1.bias', 'layer2.1.running_mean', 'layer2.1.running_var', 'layer2.1.num_batches_tracked', 'layer3.0.weight', 'layer3.0.bias', 'layer3.1.weight', 'layer3.1.bias', 'layer3.1.running_mean', 'layer3.1.running_var', 'layer3.1.num_batches_tracked', 'fc.weight', 'fc.bias'])

In [5]:
for name,param in net.named_parameters():
    if param.requires_grad and 'layer1' in name:
        param.requires_grad = False
    if param.requires_grad and 'layer2' in name:
        param.requires_grad = False

In [6]:
# home directory + datasets folder
path = "/root/EEG_Model/dataset/finetune_EEG/"
#subject to run
#runs = [3,5,7,9]
runs = [4,6,8,10]
subjects = [8]
#recorded eeg class
eeg = EEG(path, subjects, runs)
raw=eeg.data_to_raw()

print("Raw done")
# apply filter 
raw=raw.notch_filter([50,75,100])
raw=raw.filter( 8,14, method='fir', verbose=20)
print("Filter done")

epochs=eeg.epochs(raw,tmin=0,tmax=7,baseline=(0,2))
#X = X[:, :,np.newaxis,:]
X, y = eeg.get_X_y(epochs)
print(X.shape)
print(y)

/root/EEG_Model/dataset/finetune_EEG/


/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S008/S008R03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S008/S008R05.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S008/S008R07.fif) does not conform to MNE naming conventions. A

['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'STIM MARKERS']
250.0
['C3', 'C4', 'STIM MARKERS']
['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8', 'STIM MARKERS']
250.0
['C3', 'C4', 'STIM MARKERS']
Raw done
Filtering raw data in 4 contiguous segments
Setting up band-pass filter from 8 - 14 Hz



/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S008/S008R05.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S008/S008R07.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif( path_file , preload=True, verbose='WARNING' )
/root/EEG_Model/common.py:255: RuntimeWarning: This filename (/root/EEG_Model/dataset/finetune_EEG/S008/S008R09.fif) does not conform to MNE naming conventions. A

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 413 samples (1.652 sec)

Filter done
240 events found
Event IDs: [1 2 4]
Not setting metadata
Not setting metadata
80 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 80 events and 1751 original time points ...
4 bad epochs dropped
(76, 2, 1751)
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0]


/root/EEG_Model/common.py:280: RuntimeWarning: No matching events found for 3 (event id 3)
  epochs = mne.Epochs(


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

print('train size',X_train.shape, y_train.shape)
print('Test size',X_test.shape, y_test.shape)

train size (53, 2, 1751) (53,)
Test size (23, 2, 1751) (23,)


In [8]:
batch_size = 1751

train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)

num_step =math.ceil(len(train_loader.dataset) / batch_size)

In [9]:
import wandb
wandb.login()


wand = wandb.init(
        
      # Set the project where this run will be logged
      project="Motor-Imagery", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"CNN_SunsunFine_S8_EX", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0000001,
      "architecture": "CNN",
      "dataset": "Nutapol T.",
      "epochs": 300000,
      "weightname":"S8_SunsunFine_MI",
      "num_step_per_epoch" : num_step, 
        
      }
    )

config = wand.config
print(config.num_step_per_epoch)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: newturno. Use `wandb login --relogin` to force relogin


1


In [10]:
from common import train
config = wand.config
optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()
device = 'cuda'

train_loss,valid_loss,train_accuracy,valid_accuracy =train(
    model = net,
    loader_train = train_loader,
    loader_test = test_loader,
    vail_loader = None,
    optimizer = optimizer  ,
    criterion = criterion ,
    device = 'cuda',
    wand = wand
)

wandb.alert(
            title='Finish',
            text=f'Finishing training',
        )

Epoch 1/300000, Tr Loss: 2.3030, Tr Acc: 58.4906, Val Loss: 6.8469, Val Acc: 52.1739
Epoch 101/300000, Tr Loss: 2.1263, Tr Acc: 56.6038, Val Loss: 2.7438, Val Acc: 52.1739
Epoch 201/300000, Tr Loss: 2.0105, Tr Acc: 62.2642, Val Loss: 2.7722, Val Acc: 52.1739
Epoch 301/300000, Tr Loss: 2.0464, Tr Acc: 56.6038, Val Loss: 2.7935, Val Acc: 52.1739
Epoch 401/300000, Tr Loss: 1.8605, Tr Acc: 56.6038, Val Loss: 2.8118, Val Acc: 52.1739
Epoch 501/300000, Tr Loss: 2.0534, Tr Acc: 56.6038, Val Loss: 2.8282, Val Acc: 52.1739
Epoch 601/300000, Tr Loss: 1.9489, Tr Acc: 60.3774, Val Loss: 2.8446, Val Acc: 52.1739
Epoch 701/300000, Tr Loss: 1.9140, Tr Acc: 56.6038, Val Loss: 2.8483, Val Acc: 52.1739
Epoch 801/300000, Tr Loss: 1.6967, Tr Acc: 60.3774, Val Loss: 2.8609, Val Acc: 52.1739
Epoch 901/300000, Tr Loss: 1.7725, Tr Acc: 56.6038, Val Loss: 2.8659, Val Acc: 52.1739
Epoch 1001/300000, Tr Loss: 1.6981, Tr Acc: 60.3774, Val Loss: 2.8699, Val Acc: 52.1739
Epoch 1101/300000, Tr Loss: 1.6007, Tr Acc: 

KeyboardInterrupt: 